# Pruebas Dinamicas de Agente Industrial

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import random
import shutil
import numpy as np
import matplotlib.image as mpimg
import cv2
from pathlib import Path
import shutil
from typing import List, Tuple, Dict, Optional
import logging

# Agrego path de root
ROOT_PATH = str(Path.cwd().parent.parent)
if ROOT_PATH not in sys.path:
    sys.path.append(ROOT_PATH)

# Path de las imagenes
IMG_DATA_PATH = Path(ROOT_PATH + "/data/raw/images/all")

from src.vision.classifier import ImageClassifier

logging.basicConfig(level=logging.INFO)

In [2]:
# Configuracion de matplotlib y paths
plt.rcParams['figure.figsize'] = [16, 8]
plt.rcParams['figure.dpi'] = 100

IMG_DATA_PATH = Path(ROOT_PATH + "/data/raw/images/all")
print(IMG_DATA_PATH)

/home/german/Documents/ia1_2025/TrabajoFinal/trabajo-final-IA1/data/raw/images/all


In [3]:
def analyze_dataset_structure():
    """Analiza la estructura actual del dataset"""
    print("🔍 ANALIZANDO ESTRUCTURA DEL DATASET")
    
    # Encontrar todas las clases (subdirectorios de train)
    classes = [d.name for d in IMG_DATA_PATH.iterdir() if d.is_dir()]
    print(f"🏷️  Clases encontradas: {classes}")
    
    if not classes:
        print("❌ No se encontraron subdirectorios de clases en 'train/'")
        print("   La estructura debería ser: data/raw/images/train/tornillos/, etc.")
        return {}, []
    
    stats = {}
    total_images = 0
    
    # Extensiones de imagen válidas
    valid_extensions = {'.jpg', '.jpeg', '.JPG', '.JPEG', '.png', '.PNG'}
    
    for class_name in classes:
        class_path = IMG_DATA_PATH / class_name

        # Contar imágenes usando iterdir() en lugar de glob
        image_files = []
        for file_path in class_path.iterdir():
            if file_path.is_file() and file_path.suffix in valid_extensions:
                image_files.append(file_path)
        
        stats[class_name] = len(image_files)
        total_images += len(image_files)
        
        print(f"   {class_name}: {len(image_files)} imágenes")
    
    print(f"📊 Total de imágenes: {total_images}")
    return stats, classes

In [4]:
def plot_image(image: np.ndarray, title: str = "Imagen"):
    """
    Muestra una imagen usando matplotlib.

    Args:
        image (np.ndarray): Imagen a mostrar.
        title (str): Título de la imagen.
    """
    plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
    plt.axis('off')
    plt.title(title)
    plt.show()

In [ ]:
def train_best_model(attempts=3, min_accuracy=0.85):
    """
    Intenta entrenar el modelo 'attempts' veces.
    Solo guarda si supera 'min_accuracy' o si es el mejor intento hasta ahora.
    """
    data_path = IMG_DATA_PATH
    
    best_accuracy = 0.0
    
    print(f"{'='*60}")
    print(f"🚀 INICIANDO PROTOCOLO DE ENTRENAMIENTO (Intentos: {attempts})")
    print(f"{'='*60}")

    classifier = ImageClassifier()

    for i in range(1, attempts + 1):
        print(f"\n🔄 INTENTO {i}/{attempts}...")
        
        # Entrenamos (Esto divide 80/20, entrena KMeans y valida)
        # NOTA: ImageClassifier.train() guarda automáticamente. 
        # Para este script avanzado, confiamos en que guarde el último, 
        # pero aquí validamos si "vale la pena" celebrar.
        accuracy = classifier.train(data_path)
        
        print(f"   🎯 Resultado Intento {i}: {accuracy:.2%} de precisión")
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            print("   ✨ ¡Nuevo mejor modelo encontrado!")
            
            # Si ya es muy bueno, paramos antes
            if best_accuracy >= 0.98:
                print("   🏆 Precisión excelente alcanzada. Terminando temprano.")
                break
        else:
            print("   📉 Este intento no mejoró el anterior.")

    print(f"\n{'='*60}")
    print(f"🏁 FIN DEL PROCESO")
    print(f"   Mejor precisión obtenida: {best_accuracy:.2%}")
    
    if best_accuracy < min_accuracy:
        print("   ⚠️ ADVERTENCIA: El modelo final no alcanzó la precisión mínima deseada.")
    else:
        print("   ✅ Modelo guardado y listo para producción.")
    print(f"{'='*60}")

if __name__ == "__main__":
    # Ajusta min_accuracy según qué tan exigente quieras ser
    train_best_model(attempts=5, min_accuracy=0.80)

INFO:src.vision.preprocessor:✅ ImagePreprocessor inicializado correctamente
INFO:src.vision.segmentator:✅ Segmentator inicializado correctamente
INFO:src.vision.classifier:🏆 Iniciando Torneo de Entrenamiento (10 rondas)...
INFO:src.vision.classifier:Datos cargados. Entrenamiento: 50, Validación: 15.


🚀 INICIANDO PROTOCOLO DE ENTRENAMIENTO (Intentos: 5)

🔄 INTENTO 1/5...


INFO:src.vision.kmeans:🧠 Iniciando K-Means con K=4 sobre 51 muestras.
INFO:src.vision.kmeans:✅ Entrenamiento finalizado. Mejor inercia: 22.8997
INFO:src.vision.classifier:   ROUND 1: Precisión 93.33%
INFO:src.vision.classifier:   ✨ ¡Nuevo Campeón! (93.33%) Guardado.
INFO:src.vision.classifier:Datos cargados. Entrenamiento: 50, Validación: 15.


💾 Sistema de visión guardado.


INFO:src.vision.kmeans:🧠 Iniciando K-Means con K=4 sobre 51 muestras.
INFO:src.vision.kmeans:✅ Entrenamiento finalizado. Mejor inercia: 23.1325
INFO:src.vision.classifier:   ROUND 2: Precisión 100.00%
INFO:src.vision.classifier:   ✨ ¡Nuevo Campeón! (100.00%) Guardado.
INFO:src.vision.classifier:🏁 Torneo finalizado. Ganador con: 100.00%


💾 Sistema de visión guardado.
   🎯 Resultado Intento 1: 100.00% de precisión
   ✨ ¡Nuevo mejor modelo encontrado!
   🏆 Precisión excelente alcanzada. Terminando temprano.

🏁 FIN DEL PROCESO
   Mejor precisión obtenida: 100.00%
   ✅ Modelo guardado y listo para producción.


## OTROS

### Dividir el dataset en Train y Test

In [6]:
import random
import shutil
from pathlib import Path

def split_dataset():
    """
    Divide el dataset 70-30 entre train y test desde el directorio all
    """

    # Crear directorio train y test si no existen
    TRAIN_PATH.mkdir(parents=True, exist_ok=True)
    TEST_PATH.mkdir(parents=True, exist_ok=True)
    
    # Para cada clase en train
    for class_dir in ALL_PATH.iterdir():
        if class_dir.is_dir():
            class_name = class_dir.name
            print(f"Procesando clase: {class_name}")
            
            # Obtener todas las imágenes
            image_files = list(class_dir.glob("*.*"))
            random.shuffle(image_files)
            
            # Calcular splits
            split_index = int(0.7 * len(image_files))
            train_images = image_files[:split_index]
            test_images = image_files[split_index:]
            
            # Crear directorios de clase en test
            train_class_dir = TRAIN_PATH / class_name
            test_class_dir = TEST_PATH / class_name
            train_class_dir.mkdir(exist_ok=True)
            test_class_dir.mkdir(exist_ok=True)
            
            # Copiar imágenes a train
            for image in train_images:
                shutil.copy2(str(image), str(train_class_dir / image.name))

            # Copiar imágenes a test
            for image in test_images:
                shutil.copy2(str(image), str(test_class_dir / image.name))
            
            print(f"  Train: {len(train_images)} imágenes")
            print(f"  Test: {len(test_images)} imágenes")
            print(f"  Total: {len(image_files)} imágenes")

In [7]:
# Solo ejecutar 1 vez
#split_dataset()